In [1]:
# Import necessary libraries and initialize a SparkSession
from pyspark.sql import SparkSession

In [2]:
# Initialize a Spark session
spark = SparkSession.builder.appName("Question_2").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/12 11:26:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load the necessary datasets
reviews_df = spark.read.csv("Data/reviews_translated.csv", header=True, inferSchema=True)
order_item_df = spark.read.csv("Data/olist_order_items_dataset.csv", header=True, inferSchema=True)
product_df = spark.read.csv("Data/olist_products_dataset.csv", header=True, inferSchema=True)

In [4]:
reviews_df.show()

+---+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|_c0|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+---+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+
|  0|7bc2406110b926393...|73fc7af87114b3971...|           4|                null|                  null| 2018-01-18 00:00:00|    2018-01-18 21:46:59|
|  1|80e641a11e56f04c1...|a548910a1c6147796...|           5|                null|                  null| 2018-03-10 00:00:00|    2018-03-11 03:05:13|
|  2|228ce5500dc1d8e02...|f9e4b658b201a9f2e...|           5|                null|                  null| 2018-02-17 00:00:00|    2018-02-18 14:36:24|
|  3|e64fb393e7b32834b...|658677c97b385a9be...|           5|                null|  I received it wel

23/09/12 11:26:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
 Schema: _c0, review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
Expected: _c0 but found: 
CSV file: file:///home/user/Documents/Fusemachines/Spark_FInal_Projetc/Data/reviews_translated.csv


In [5]:
reviews_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)



In [6]:
reviews_df=reviews_df.withColumn("review_score", reviews_df["review_score"].cast("int"))

In [7]:
reviews_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)



In [8]:
from pyspark.sql import functions as f

In [11]:
count_no_comment = reviews_df.filter(f.col("review_comment_message") == "no comment").count()

print("Count of 'No comment':", count_no_comment)

Count of 'No comment': 61809


In [12]:
print(reviews_df.count())

102785


In [13]:
reviews_df =reviews_df.na.drop()
print(reviews_df.count())

23/09/12 11:27:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
 Schema: _c0, review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
Expected: _c0 but found: 
CSV file: file:///home/user/Documents/Fusemachines/Spark_FInal_Projetc/Data/reviews_translated.csv


96515


In [14]:
count_no_comment = reviews_df.filter(f.col("review_comment_message") == "no comment").count()

print("Count of 'No comment':", count_no_comment)

23/09/12 11:27:02 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
 Schema: _c0, review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
Expected: _c0 but found: 
CSV file: file:///home/user/Documents/Fusemachines/Spark_FInal_Projetc/Data/reviews_translated.csv


Count of 'No comment': 58274


In [15]:
reviews_df.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in reviews_df.columns]
   ).show()

23/09/12 11:27:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
 Schema: _c0, review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
Expected: _c0 but found: 
CSV file: file:///home/user/Documents/Fusemachines/Spark_FInal_Projetc/Data/reviews_translated.csv


+---+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|_c0|review_id|order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|
+---+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+
|  0|        0|       0|           0|                   0|                     0|                   0|                      0|
+---+---------+--------+------------+--------------------+----------------------+--------------------+-----------------------+



### Perform sentiment analysis on the comment

In [16]:
from textblob import TextBlob
from pyspark.sql.types import StringType

In [17]:
def analyze_sentiment(text):
    if text.lower() != "no comment":
        analysis = TextBlob(text)
        polarity = analysis.sentiment.polarity
        return polarity
    return None  # Return None for "no comment" comments

In [18]:
sentiment_udf = f.udf(analyze_sentiment, StringType())

In [20]:
reviews_df = reviews_df.withColumn("sentiment_score", sentiment_udf(reviews_df["review_comment_message"]))
# reviews_df.show()
reviews_df=reviews_df.withColumn("sentiment_score", reviews_df["sentiment_score"].cast("float"))
# Filter out rows with sentiment scores (exclude "no comment" comments)
filtered_reviews_df = reviews_df.filter(reviews_df["sentiment_score"].isNotNull())

# Show the filtered DataFrame with sentiment scores
filtered_reviews_df.show()
filtered_reviews_df.printSchema()

23/09/12 11:27:51 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
 Schema: _c0, review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
Expected: _c0 but found: 
CSV file: file:///home/user/Documents/Fusemachines/Spark_FInal_Projetc/Data/reviews_translated.csv


+---+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+---------------+
|_c0|           review_id|            order_id|review_score|review_comment_title|review_comment_message|review_creation_date|review_answer_timestamp|sentiment_score|
+---+--------------------+--------------------+------------+--------------------+----------------------+--------------------+-----------------------+---------------+
|  3|e64fb393e7b32834b...|658677c97b385a9be...|           5|          no comment|  I received it wel...| 2017-04-21 00:00:00|    2017-04-21 22:02:06|            0.0|
|  4|f7c4243c7fe1938f1...|8e6bfb81e283fa7e4...|           5|          no comment|  Congratulations L...| 2018-03-01 00:00:00|    2018-03-02 10:26:53|      0.6666667|
|  9|8670d52e15e00043a...|b9bf720beb4ab3728...|           4|         I recommend|  efficient device....| 2018-05-22 00:00:00|    2018-05-23 16:45:47|            0.0|
| 12

In [21]:
correlation = filtered_reviews_df.select(f.corr("sentiment_score", "review_score")).first()[0]
print("Correlation between sentiment_score and review_score:", correlation)

23/09/12 11:28:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
 Schema: _c0, review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
Expected: _c0 but found: 
CSV file: file:///home/user/Documents/Fusemachines/Spark_FInal_Projetc/Data/reviews_translated.csv


Correlation between sentiment_score and review_score: 0.536884175049079


In [22]:
filtered_reviews_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: integer (nullable = true)
 |-- review_comment_title: string (nullable = false)
 |-- review_comment_message: string (nullable = false)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)
 |-- sentiment_score: float (nullable = true)



In [26]:
results_df = filtered_reviews_df.join(order_item_df, "order_id", "inner")
joined_df= results_df.join(product_df, "product_id", "inner")
joined_df = joined_df.select("order_id", "review_id","review_score", "review_comment_message","product_category_name","sentiment_score")

joined_df.show()

23/09/12 11:29:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
 Schema: _c0, review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
Expected: _c0 but found: 
CSV file: file:///home/user/Documents/Fusemachines/Spark_FInal_Projetc/Data/reviews_translated.csv


+--------------------+--------------------+------------+----------------------+---------------------+---------------+
|            order_id|           review_id|review_score|review_comment_message|product_category_name|sentiment_score|
+--------------------+--------------------+------------+----------------------+---------------------+---------------+
|00010242fe8c5a6d1...|97ca439bc427b48bc...|           5|  Perfect, product ...|           cool_stuff|            1.0|
|000229ec398224ef6...|0c5b33dea94867d1a...|           5|  It arrived ahead ...|     moveis_decoracao|          0.375|
|00042b26cf59d7ce6...|940144190dcba6351...|           5|  I liked it becaus...|   ferramentas_jardim|            0.6|
|00061f2a7bc09da83...|7e92c32ae297ef950...|           5|  The product box c...|         beleza_saude|        0.40625|
|00063b381e2406b52...|5595c192e3c1566ab...|           5|  I was a little sa...| fashion_bolsas_e_...|      -0.246875|
|0006ec9db01a64e59...|6322c405c0f34bf3a...|           5|

In [24]:


joined_df = joined_df.withColumn(
    "sentiment",
    f.when(joined_df["sentiment_score"] > 0, "positive")
    .when(joined_df["sentiment_score"] == 0, "neutral")
    .otherwise("negative")
)

joined_df.show()


23/09/12 11:28:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
 Schema: _c0, review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
Expected: _c0 but found: 
CSV file: file:///home/user/Documents/Fusemachines/Spark_FInal_Projetc/Data/reviews_translated.csv


+--------------------+--------------------+----------------------+---------------------+---------------+---------+
|            order_id|           review_id|review_comment_message|product_category_name|sentiment_score|sentiment|
+--------------------+--------------------+----------------------+---------------------+---------------+---------+
|00010242fe8c5a6d1...|97ca439bc427b48bc...|  Perfect, product ...|           cool_stuff|            1.0| positive|
|000229ec398224ef6...|0c5b33dea94867d1a...|  It arrived ahead ...|     moveis_decoracao|          0.375| positive|
|00042b26cf59d7ce6...|940144190dcba6351...|  I liked it becaus...|   ferramentas_jardim|            0.6| positive|
|00061f2a7bc09da83...|7e92c32ae297ef950...|  The product box c...|         beleza_saude|        0.40625| positive|
|00063b381e2406b52...|5595c192e3c1566ab...|  I was a little sa...| fashion_bolsas_e_...|      -0.246875| negative|
|0006ec9db01a64e59...|6322c405c0f34bf3a...|  Excellent online ...|      cama_mes

In [25]:
joined_df.select([f.count(f.when(f.isnan(c) | f.col(c).isNull(), c)).alias(c) for c in joined_df.columns]).show()

23/09/12 11:28:46 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
 Schema: _c0, review_id, order_id, review_score, review_comment_title, review_comment_message, review_creation_date, review_answer_timestamp
Expected: _c0 but found: 
CSV file: file:///home/user/Documents/Fusemachines/Spark_FInal_Projetc/Data/reviews_translated.csv


+--------+---------+----------------------+---------------------+---------------+---------+
|order_id|review_id|review_comment_message|product_category_name|sentiment_score|sentiment|
+--------+---------+----------------------+---------------------+---------------+---------+
|       0|        0|                     0|                  653|              0|        0|
+--------+---------+----------------------+---------------------+---------------+---------+

